In [32]:
import dhlab.nbtext as nb
import dhlab.module_update as mu
import json
import requests
import os
import pandas as pd
#import Levenshtein as lev
import ast
mu.css()

In [2]:
AVSNITTMAPPE = "Avsnitt/"

In [3]:
#!pip install fuzzywuzzy

In [4]:
#!pip install python-Levenshtein

In [5]:
from fuzzywuzzy import fuzz

In [6]:
def get_paragraphs(urn, paras):
    """Return paragraphs for urn"""
    param = dict()
    param['paragraphs'] = paras
    param['urn'] = urn
    r = requests.get("https://api.nb.no/ngram/paragraphs", json=param)
    return r.json()

In [7]:
def get_para(para_list):
    import ast
    paras = []
    for p in para_list:
        paras.append(ast.literal_eval(p))
    urns = [x[0] for x in paras]
    paradict = {u:[x[1] for x in paras if x[0] == u] for u in urns}
    paragraphs = dict()
    for u in paradict:
        paragraphs[u] = get_paragraphs(u, paradict[u])
    return paragraphs

def get_paras_as_list(para_list):
    import ast
    paras = []
    for p in para_list:
        paras.append(ast.literal_eval(p))
    urns = [x[0] for x in paras]
    paradict = {u:[x[1] for x in paras if x[0] == u] for u in urns}
    paragraphs = dict()
    for u in paradict:
        paragraphs[u] = get_paragraphs(u, paradict[u])
    return paragraphs

In [8]:
def find_theme(title, themes):
    from collections import Counter
    dist = Counter()
    for th in themes:
        dist[th] = fuzz.ratio(th, title)
    x = dist.most_common(1)[0][0]
    if dist[x] > 60:
        res = x
    else:
        res = ""
    return res

In [9]:
def show_paras(title, themes):
    avsnitt = json.load(open(title))
    paras = get_para(avsnitt)
    theme = find_theme(title, themes)
    words = [w for hyperlexeme in themes[theme] for w in themes[theme][hyperlexeme]]
    ptext = "<h2>" + theme + "</h2>"
    ptext += "<h3" + ', '.join(themes[theme].keys()) + "</h3>"
    for urn in paras:
        ptext += "<h4> <a href='" + link(urn) +"'" + "target=_>" + ', '.join([str(x) for x in nb.metadata(urn)[0][1:4]]) + "</a>"
        for paranum in paras[urn]:
            ptext += "<h4>" + paranum + '</h4>'
            modified = paras[urn][paranum]
            #print(modified)
            for w in words:
                #print(w)
                modified = modified.replace(" "+w+" ", " <b>" + w + "</b> ")
            ptext += modified + '\n\n'
    return ptext

def fetch_paras(title, themes):
    avsnitt = json.load(open(title))
    df = pd.DataFrame(avsnitt).fillna(0).transpose()
    sums = df.sum(axis=1)
    counts = df[df > 0].count(axis = 1)
    df['sum'] = sums
    df['count'] = counts
    paras = []
    for x in df.sort_values(by='count', ascending=False).index:
        (u, p) = ast.literal_eval(x)
        paras.append((u, (get_paragraphs(u, [p]))))
    theme = find_theme(title, themes)
    words = [w for hyperlexeme in themes[theme] for w in themes[theme][hyperlexeme]]
    ptext = "<h2>" + theme + "</h2>"
    ptext += "<h3" + ', '.join(themes[theme].keys()) + "</h3>"
    # paras = [(urn, {parnum: para})]
    for urn in paras:
        ptext += "<h4> <a href='" + link(urn[0]) +"'" + "target=_>" + ', '.join([str(x) for x in nb.metadata(urn[0])[0][1:4]]) + "</a>"
        parnum = list(urn[1].keys())[0]
        ptext += "<h4>" + parnum + '</h4>'
        modified = urn[1][parnum]
        #print(modified)
        for w in words:
            #print(w)
            modified = modified.replace(" "+w+" ", " <b>" + w + "</b> ")
        ptext += modified + '\n\n'
    return ptext



def sort_paras(title, themes):
    avsnitt = json.load(open(title))
    return avsnitt

In [34]:
with open('heidis_temadefinisjoner_desember.json') as f:
    themes = json.load(f)

In [35]:
link = lambda x: "https://urn.nb.no/URN:NBN:no-nb_digibok_" + str(x)

In [36]:
print(len(themes))
for th in themes:
    print(th)

23
“Det moderlige(1)/ The motherly”
“Det moderlige(2)/ The motherly”
“Det moderlige(3)/ The motherly”
“Det moderlige(4)/ The motherly”
“Barmhjertighed (1)/charity”
“Barmhjertighed (2)/charity”
“Skjønne piker/ beautiful girls”
“Moderlige inntrykk/Motherly impressions”
“Kvinnen i samfunnet/Woman in society”
“Pigelivets begrensninger/the restrictions of a girl’s life”
“Dansens metafor/the metaphor of dance”
“Kvindekjønnets last (1)/The vice of the female sex”
“Kvindekjønnets last (2)/The vice of the female sex”
“Woman’s happiness”
“Kvinnens behov/woman’s needs”
“En glødende sjels livsvilkår/ the life conditions of a fiery soul”
“Kvindens frihed/Woman’s freedom”
“Guds skapelse av mennesket (1)/God’s creation of man”
“Guds skapelse av mennesket (2)/God’s creation of man”
“Guds skapelse av kvinden / God’s creation of woman”
“Holdning til samfunnet/ Attitude towards society”
“Kvindens underskastelse/ Woman’s submission”
“Kvindens menneskeverd/ Woman’s dignity”


In [37]:
#themes

In [38]:
#themes[find_theme('forfatter og religion', themes)]

In [39]:
import os
r,d,f = next(os.walk("Avsnitt/"))
jsons = [fs for fs in f if fs.endswith('.json')]
files = {f:find_theme(f, themes) for f in jsons if find_theme(f, themes) != ''}

In [40]:
print(len(files))
files

23


{'Barmhjertighed 1charity.json': '“Barmhjertighed (1)/charity”',
 'Barmhjertighed 2charity.json': '“Barmhjertighed (2)/charity”',
 'Dansens metaforthe metaphor of dance.json': '“Dansens metafor/the metaphor of dance”',
 'Det moderlige1 The motherly.json': '“Det moderlige(1)/ The motherly”',
 'Det moderlige2 The motherly.json': '“Det moderlige(2)/ The motherly”',
 'Det moderlige3 The motherly.json': '“Det moderlige(3)/ The motherly”',
 'Det moderlige4 The motherly.json': '“Det moderlige(4)/ The motherly”',
 'En glødende sjels livsvilkår the life conditions of a fiery soul.json': '“En glødende sjels livsvilkår/ the life conditions of a fiery soul”',
 'Guds skapelse av kvinden  Gods creation of woman.json': '“Guds skapelse av kvinden / God’s creation of woman”',
 'Guds skapelse av mennesket 1Gods creation of man.json': '“Guds skapelse av mennesket (1)/God’s creation of man”',
 'Guds skapelse av mennesket 2Gods creation of man.json': '“Guds skapelse av mennesket (2)/God’s creation of man”'

In [41]:
for f in files:
    print(f.split('.json')[0])

Barmhjertighed 1charity
Barmhjertighed 2charity
Dansens metaforthe metaphor of dance
Det moderlige1 The motherly
Det moderlige2 The motherly
Det moderlige3 The motherly
Det moderlige4 The motherly
En glødende sjels livsvilkår the life conditions of a fiery soul
Guds skapelse av kvinden  Gods creation of woman
Guds skapelse av mennesket 1Gods creation of man
Guds skapelse av mennesket 2Gods creation of man
Holdning til samfunnet Attitude towards society
Kvindekjønnets last 1The vice of the female sex
Kvindekjønnets last 2The vice of the female sex
Kvindens frihedWomans freedom
Kvindens menneskeverd Womans dignity
Kvindens underskastelse Womans submission
Kvinnen i samfunnetWoman in society
Kvinnens behovwomans needs
Moderlige inntrykkMotherly impressions
Pigelivets begrensningerthe restrictions of a girls life
Skjønne piker beautiful girls
Womans happiness



No sort

In [40]:
res = ""
for f in files:
    print(f)
    newfilename = f.split('.json')[0] + '_avsnitt.html'
    res = show_paras(os.path.join(AVSNITTMAPPE,f), themes) + "\n"
    with open(os.path.join(AVSNITTMAPPE, newfilename), "w", encoding='utf-8') as html:
        html.write(res)

Barmhjertighed 1charity.json
Barmhjertighed 2charity.json
Dansens metaforthe metaphor of dance.json
Det moderlige1 The motherly.json
Det moderlige2 The motherly.json
Det moderlige3 The motherly.json
Det moderlige4 The motherly.json
Kvindekjønnets last 1The vice of the female sex.json
Kvindekjønnets last 2The vice of the female sex.json
Kvinnen i samfunnetWoman in society.json
Moderlige inntrykkMotherly impressions.json
Pigelivets begrensningerthe restrictions of a girls life.json
Skjønne piker beautiful girls.json


Sorted paras

In [26]:
files

{'Barmhjertighed 1charity.json': '“Barmhjertighed (1)/charity”',
 'Barmhjertighed 2charity.json': '“Barmhjertighed (2)/charity”',
 'Dansens metaforthe metaphor of dance.json': '“Dansens metafor/the metaphor of dance”',
 'Det moderlige1 The motherly.json': '“Det moderlige(1)/ The motherly”',
 'Det moderlige2 The motherly.json': '“Det moderlige(2)/ The motherly”',
 'Det moderlige3 The motherly.json': '“Det moderlige(3)/ The motherly”',
 'Det moderlige4 The motherly.json': '“Det moderlige(4)/ The motherly”',
 'En glødende sjels livsvilkår the life conditions of a fiery soul.json': '“En glødende sjels livsvilkår/ the life conditions of a fiery soul”',
 'Guds skapelse av kvinden  Gods creation of woman.json': '“Guds skapelse av kvinden / God’s creation of woman”',
 'Guds skapelse av mennesket 1Gods creation of man.json': '“Guds skapelse av mennesket (1)/God’s creation of man”',
 'Guds skapelse av mennesket 2Gods creation of man.json': '“Guds skapelse av mennesket (2)/God’s creation of man”'

Take most of that in files

In [122]:
res = ""
for f in files:
    print(f)
    newfilename = f.split('.json')[0] + '_avsnitt.html'
    res = fetch_paras(os.path.join(AVSNITTMAPPE,f), themes) + "\n"
    with open(os.path.join(AVSNITTMAPPE, newfilename), "w", encoding='utf-8') as html:
        html.write(res)

Barmhjertighed 1charity.json
Barmhjertighed 2charity.json
Dansens metaforthe metaphor of dance.json
Det moderlige1 The motherly.json
Det moderlige2 The motherly.json
Det moderlige3 The motherly.json
Det moderlige4 The motherly.json
En glødende sjels livsvilkår the life conditions of a fiery soul.json
Guds skapelse av kvinden  Gods creation of woman.json
Guds skapelse av mennesket 1Gods creation of man.json
Guds skapelse av mennesket 2Gods creation of man.json
Kvindekjønnets last 1The vice of the female sex.json
Kvindekjønnets last 2The vice of the female sex.json
Kvindens frihedWomans freedom.json
Kvindens menneskeverd Womans dignity.json
Kvindens underskastelse Womans submission.json
Kvinnen i samfunnetWoman in society.json
Kvinnens behovwomans needs.json
Moderlige inntrykkMotherly impressions.json
Pigelivets begrensningerthe restrictions of a girls life.json
Skjønne piker beautiful girls.json


Process residue files: woman's happiness

In [29]:
residue = {x:files[x] for x in files if "happiness" in x}
residue

{'Womans happiness.json': '“Woman’s happiness”'}

In [33]:
res = ""
for f in residue:
    print(f)
    newfilename = f.split('.json')[0] + '_avsnitt.html'
    res = fetch_paras(os.path.join(AVSNITTMAPPE,f), themes) + "\n"
    with open(os.path.join(AVSNITTMAPPE, newfilename), "w", encoding='utf-8') as html:
        html.write(res)

Womans happiness.json


In [43]:
residue = {x:files[x] for x in files if "Attitude" in x}
residue

{'Holdning til samfunnet Attitude towards society.json': '“Holdning til samfunnet/ Attitude towards society”'}

In [44]:
res = ""
for f in residue:
    print(f)
    newfilename = f.split('.json')[0] + '_avsnitt.html'
    res = fetch_paras(os.path.join(AVSNITTMAPPE,f), themes) + "\n"
    with open(os.path.join(AVSNITTMAPPE, newfilename), "w", encoding='utf-8') as html:
        html.write(res)

Holdning til samfunnet Attitude towards society.json


# End of processing

In [30]:
res = dict()
for f in residue:
    print(f)
    res[f] = sort_paras(os.path.join(AVSNITTMAPPE,f), themes)
for f in res:
    newfilename = f.split('.json')[0] + '_avsnitt.html'
    

Womans happiness.json


In [48]:
import pandas as pd

In [50]:
df = {f:pd.DataFrame(res[f]) for f in res}

In [92]:
df['Barm2'] = df['Barmhjertighed 2charity.json'].fillna(0).transpose()

In [93]:
sums = df['Barm2'][df['Barm2'] > 0].sum(axis=1)
counts = df['Barm2'][df['Barm2'] > 0].count(axis=1)

In [95]:
df['Barm2']['sum'] = sums

In [96]:
df['Barm2']['count'] = counts

In [98]:
df['Barm2'].sort_values(by='count', ascending=False)

,Barmhjertighed_,Be_,Gave_,Moderbryst_,Rest_,sum,count
"('2008100603020', 1122)",2.0,0.0,1.0,2.0,1.0,6.0,4
"('2009092203019', 8305)",0.0,1.0,0.0,1.0,1.0,3.0,3
"('2016022548049', 2573)",0.0,1.0,0.0,1.0,1.0,3.0,3
"('2015010648111', 494)",0.0,1.0,0.0,2.0,2.0,5.0,3
"('2015052748022', 494)",0.0,1.0,1.0,0.0,1.0,3.0,3
"('2008100603020', 4356)",0.0,1.0,7.0,0.0,2.0,10.0,3
"('2015052748022', 1853)",0.0,2.0,0.0,1.0,1.0,4.0,3
"('2008111303008', 5200)",0.0,0.0,1.0,1.0,3.0,5.0,3
"('2008111303008', 5206)",0.0,1.0,2.0,0.0,1.0,4.0,3
"('2008100603020', 4310)",0.0,1.0,0.0,1.0,1.0,3.0,3


In [109]:
paras = []
for x in df['Barm2'].sort_values(by='count', ascending=False).index[:20]:
    (u, p) = ast.literal_eval(x)
    paras.append((u, (get_paragraphs(u, [p]))))
print(paras)
                

[('2008100603020', {'1122': '40. Bjergegnen , til en Stad i luda . " Og hun kom i Zacharias \' Hus , 41. og hilsede , Elisabeth . " Og det begav sig , der Elisabeth Horte Marice Hilsen , sprang Fosteret i hendes Liv ; og Elisabeth blcv fyldt 42. med den Hellig-Aand , " Og raabte med hoi Nost , og sagde : velsignet er du iblandt Qvinderne , og velsignet er dit Livs Frugt ! 43. " Og hvorfra kommer mig det , at min Herres Moder kommer til 44. mig ? " Thi se , der din Hilsens Nost kom mig til Oren , sprang 45. Fosteret i mit Liv med Fryd . " Og salig er hun , som troede ; thi 46. det skal fuldkommes , som hende er sagt af Herren . " Og Maria 47. sagde : min Sjel ofthoier Herren , " Og min Aand fryder sig i 48. Gud , min Frelser , " Fordi han haver seet til sin Tjenerindcs Ninghed . Thi se ! nu herefter stulle alle Slcegter prise mig salig . 49. " Thi han haver gjort store Ting imod mig , han , som er mcegtig , 50. og hvis Navn er helligt . " Og hans Barmhjertighed varer fra 51. Slcegt til 

In [110]:
for p in paras:
    print(p)

('2008100603020', {'1122': '40. Bjergegnen , til en Stad i luda . " Og hun kom i Zacharias \' Hus , 41. og hilsede , Elisabeth . " Og det begav sig , der Elisabeth Horte Marice Hilsen , sprang Fosteret i hendes Liv ; og Elisabeth blcv fyldt 42. med den Hellig-Aand , " Og raabte med hoi Nost , og sagde : velsignet er du iblandt Qvinderne , og velsignet er dit Livs Frugt ! 43. " Og hvorfra kommer mig det , at min Herres Moder kommer til 44. mig ? " Thi se , der din Hilsens Nost kom mig til Oren , sprang 45. Fosteret i mit Liv med Fryd . " Og salig er hun , som troede ; thi 46. det skal fuldkommes , som hende er sagt af Herren . " Og Maria 47. sagde : min Sjel ofthoier Herren , " Og min Aand fryder sig i 48. Gud , min Frelser , " Fordi han haver seet til sin Tjenerindcs Ninghed . Thi se ! nu herefter stulle alle Slcegter prise mig salig . 49. " Thi han haver gjort store Ting imod mig , han , som er mcegtig , 50. og hvis Navn er helligt . " Og hans Barmhjertighed varer fra 51. Slcegt til S